In [0]:
%sql DESCRIBE HISTORY catalog.schema.table

1

In [0]:
from datetime import timezone

fechamento_antigo = dbutils.widgets.get("fechamento_antigo") # Parametro via Databricks Jobs
fechamento_novo= spark.sql("""
SELECT max(timestamp) AS ultimo_timestamp
FROM (DESCRIBE HISTORY catalog.schema.table)""").collect()[0]['ultimo_timestamp'].replace(tzinfo=timezone.utc).isoformat(timespec='milliseconds')

In [0]:
df = spark.sql(f"""
select *, to_date('{fechamento_novo}','yyyy-MM-dd')  as dt_fechamento_mudanca 
from catalog.schema.table TIMESTAMP AS OF '{fechamento_novo}'
union all 
select *, to_date('{fechamento_antigo}','yyyy-MM-dd')  as dt_fechamento_mudanca
from catalog.schema.table TIMESTAMP AS OF '{fechamento_antigo}'
""")

In [0]:
df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("catalog.schema.table")